<a href="https://colab.research.google.com/github/leorex819-creator/fake-news-detection-multilingual_videos/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

folders = [
    "notebooks",
    "data/raw/text", "data/raw/images", "data/raw/videos",
    "data/processed/text", "data/processed/images", "data/processed/videos",
    "src", "models/text", "models/image", "models/video",
    "results/metrics", "results/graphs", "results/predictions",
    "utils"
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)

print("✅ Professional project structure created")


✅ Professional project structure created


In [ ]:
open("data/raw/images/.gitkeep", "w").close()
open("data/raw/videos/.gitkeep", "w").close()
open("data/raw/text/.gitkeep", "w").close()


In [4]:
import os
print(os.getcwd())
print(os.listdir())


/content
['.config', 'notebooks', 'models', 'src', 'results', 'utils', 'data', 'sample_data']


In [5]:
import os

base_path = "/content/fake-news-detection-multilingual"

folders = [
    "notebooks",
    "data/raw/text",
    "data/raw/images",
    "data/raw/videos",
    "data/processed/text",
    "data/processed/images",
    "data/processed/videos",
    "src",
    "models/text",
    "models/image",
    "models/video",
    "results/metrics",
    "results/graphs",
    "results/predictions",
    "utils"
]

for folder in folders:
    os.makedirs(os.path.join(base_path, folder), exist_ok=True)

print("✅ Project structure created successfully")


✅ Project structure created successfully


In [6]:
os.chdir("/content/fake-news-detection-multilingual")
print(os.getcwd())


/content/fake-news-detection-multilingual


In [7]:
print(os.listdir("data"))
print(os.listdir("data/raw"))
print(os.listdir("data/raw/images"))


['processed', 'raw']
['videos', 'text', 'images']
[]


In [8]:
open("data/raw/images/.gitkeep", "w").close()
open("data/raw/videos/.gitkeep", "w").close()
open("data/raw/text/.gitkeep", "w").close()
